In [ ]:
import pandas as pd
import numpy as np

In [ ]:
weather_df=pd.read_csv('..\Data Files\CSV files\weatherAUS.csv')

In [ ]:
weather_df.info()
# We need to drop all rows with missing values on rain today, rain tomorrow as this can affect our model. Reason is that one is our target variable and the other is closely related to it.
weather_df.dropna(subset= ['RainTomorrow'],inplace=True)

#Declaring inputs and outputs
input_cols=(weather_df.columns)[1:-1]
target_col='RainTomorrow'

#Declaring numerical cols and categorical cols
numeric_cols=weather_df[input_cols].select_dtypes(include=np.number).columns.tolist()
categorical_cols=weather_df[input_cols].select_dtypes('object').columns.tolist()

# Check for missing values and Perform imputation
weather_df[numeric_cols].isna().sum()

# Check for missing values and Perform imputation
weather_df[numeric_cols].isna().sum()
#Imputing our data
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='mean')
imputer.fit(weather_df[numeric_cols])
# Divide/Split the data to train,validation and test df frames
year=pd.to_datetime(weather_df.Date).dt.year

train_df=weather_df[year<2015]
val_df=weather_df[year==2015]
test_df=weather_df[year>2015]

# Divide/Split the data to train,validation and test df frames
year=pd.to_datetime(weather_df.Date).dt.year

train_df=weather_df[year<2015]
val_df=weather_df[year==2015]
test_df=weather_df[year>2015]

train_inputs=train_df[input_cols].copy()
train_targets=train_df[target_col].copy
val_inputs=val_df[input_cols].copy()
val_targets=val_df[target_col].copy()
test_inputs=test_df[input_cols].copy()
test_targets=test_df[target_col].copy()
#To fill the columns with imputed statistics..
train_inputs[numeric_cols]=imputer.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols]=imputer.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols]=imputer.transform(test_inputs[numeric_cols])
#To view imputer statistics
list(imputer.statistics_)

# Check for missing values
val_inputs[numeric_cols].isna().sum()
#Scaling Numerical Values
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler().fit(weather_df[numeric_cols])
train_inputs[numeric_cols]=scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols]=scaler.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols]=scaler.transform(test_inputs[numeric_cols])
#Dealing with missing values
train_inputs[categorical_cols]=train_inputs[categorical_cols].fillna('Unknown')
val_inputs[categorical_cols]=val_inputs[categorical_cols].fillna('Unknown')
test_inputs[categorical_cols]=test_inputs[categorical_cols].fillna('Unknown')
#check out the no of unique values in the categorical columns 
weather_df[categorical_cols].nunique()

#check out the no of unique values in the categorical columns 
weather_df[categorical_cols].nunique()
#Encoding Categorical Columns
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(sparse=False,handle_unknown='ignore')
encoder.fit(weather_df[categorical_cols])

#We generate columns for each individual category using using get_feature_names
encoded_cols=list(encoder.get_feature_names(categorical_cols))
print(encoded_cols)
#We generate columns for each individual category using using get_feature_names
encoded_cols=list(encoder.get_feature_names(categorical_cols))
print(encoded_cols)
train_inputs[encoded_cols]=encoder.transform(train_inputs[categorical_cols])
val_inputs[encoded_cols]=encoder.transform(val_inputs[categorical_cols])
test_inputs[encoded_cols]=encoder.transform(test_inputs[categorical_cols])
#Dropping the textualised columns so that we are left with just numeric data
x_train=train_inputs[numeric_cols+encoded_cols]
x_val=val_inputs[numeric_cols+encoded_cols]
x_test=test_inputs[numeric_cols+encoded_cols]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_jobs=-1, random_state=42)

n_jobs allows the random forest to use mutiple parallel workers to train decision trees, and random_state=42 ensures that the we get the same results for each execution.

In [ ]:
%%time
model.fit(x_train, train_targets)

model.score(x_train, train_targets)

model.score(x_val, val_targets)


Once again, the training accuracy is almost 100%, but this time the validation accuracy is much better. In fact, it is better than the best single decision tree we had trained so far. Do you see the power of random forests?

This general technique of combining the results of many models is called "ensembling", it works because most errors of individual models cancel out on averaging. Here's what it looks like

In [ ]:
#probabilities for the predictions. 
train_probs = model.predict_proba(x_train)
train_probs

In [ ]:
#access individual decision trees using
model.estimators_[0]

In [ ]:
plt.figure(figsize=(80,20))
plot_tree(model.estimators_[0], max_depth=2, feature_names=x_train.columns, filled=True, rounded=True, class_names=model.classes_)

In [ ]:
#Hyperparameter tuning with Random forests
#We first create a basem odel in which we can compare models with tuned hyperparameters
base_model=RandomForestClassifier(random_state=42,n_jobs=1).fit(x_train,train_targets)

In [ ]:
base_train_acc=base_model.score(x_train,train_targets)
base_val_acc=base_model.score(x_val,val_targets)

In [ ]:
base_acc=base_train_acc,base_val_acc

Hyperparameters
   n_estimators=controls the number of decision trees in the random forest. Default is 100. for larger data it helps to have a greater number of estimators

ma_depth and max_leaf_nodes= pass directly to the decison treee. by default no maax_depth is specified which is why each tree has a training accuracy of 100%. Specifying max_depth reduces overfitting.

max_features={'auto','sqrt','log2'}, int or float default='auto.instead of picking alll features(columns) for every split, we can specify only a fraction of features to be randomly chosen to figure out a split.

